In [ ]:
# -*- coding: utf-8 -*-

In [ ]:
#No need to run
#i = []
#while True:
#  i.append(i)

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.spatial import distance

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

read the data

In [6]:
print('Loading data ...')
train_data = pd.read_csv('/content/drive/My Drive/Project3/train_data.csv')
test_data = pd.read_csv('/content/drive/My Drive/Project3/test_data.csv')

Loading data ...


In [7]:
train_data.head

<bound method NDFrame.head of            ID  ...                       Type
0       82577  ...              Medical Books
1       79481  ...              Medical Books
2      125658  ...                        Law
3      201053  ...                     Travel
4      204615  ...                     Travel
...       ...  ...                        ...
47194  205663  ...                     Travel
47195  195082  ...                     Travel
47196   83111  ...              Medical Books
47197  104231  ...  Health, Fitness & Dieting
47198  102812  ...  Health, Fitness & Dieting

[47199 rows x 4 columns]>

obtain and count the text data

In [8]:
train_text = list(train_data['Text'])
test_text = list(test_data['Text'])
num_of_train = len(train_text)
num_of_test = len(test_text)

labels of training data

In [9]:
train_label = list(train_data['Category'])

convert the text data to tf-idf features
using TfidfVectorizer: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [10]:
print('Extracting tfidf features ...')
corpus = train_text + test_text
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

Extracting tfidf features ...


get the number of features

In [11]:
_, num_of_features = X.shape

get train tfidf features and test tfidf features

In [12]:
X_array = X.toarray()
train_text_tfidf = X_array[:num_of_train]
test_text_tfidf = X_array[num_of_train:]

-----------------------------------------<br>
TRY YOUR OWN APPROACHES <br>
data processing: feature selection, demension reduction, data normalization, ...<br>
approaches: clustering or claasification

Example:<br>
Here I just use a very simple approach:<br>
Assume that there are N clusters, and we know the label of each data<br>
then based on the label information, we can obtain the centroid of each cluster by averaging all the data's tf-idf features that have the same label<br>
Finally, we can have the centroids of each cluster

In [ ]:
print('Training ...')
centroids = {} # key: label, value: array (sum of features belonging to the same category)
counter = {} # key: label, value: # of data belonging to the same category
for i in range(num_of_train):
    label = train_label[i]
    if label not in centroids.keys():
        centroids[label] = np.zeros(num_of_features)
        counter[label] = 0
    centroids[label] += train_text_tfidf[i]
    counter[label] += 1

Training ...


average value

In [ ]:
for k, v in centroids.items():
    centroids[k] = v/counter[k]

testing: for each data in the test set, we can calculate the euclidean distance between the data feature and each centroid<br>
the label with the samlles distance can be considered as the prediction

In [ ]:
print('Testing ...')
prediction = []
for i in range(num_of_test):
    test_i = test_text_tfidf[i]
    
    smallest_distance = 1000000000000
    for k, v in centroids.items():
        dst = distance.euclidean(test_i, v)
        if dst < smallest_distance:
            smallest_distance = dst
            pred = k
    prediction.append(pred)

Testing ...


----------------------------------------------

write to submission file

In [ ]:
df_sub = pd.read_csv('/content/drive/My Drive/Project3/sample_submission.csv', sep=',')
df_sub['Category'] = prediction
df_sub.to_csv ('my_sample_submission.csv', index = False, header=True)
print('done!')

done!


In [ ]:
df_sub

,ID,Category
0,202985,5
1,75315,1
2,75976,1
3,192608,5
4,82492,5
...,...,...
20224,84561,4
20225,125815,3
20226,101270,2
20227,72406,0
